In [1]:
# Сразу easy + normal, потому что удобно сразу все сделать с данными и ток потом обучаеться
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from catboost import CatBoostClassifier

from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('../data/singapore_airlines_reviews.csv')
data.head()

,published_date,published_platform,rating,type,text,title,helpful_votes
0,2024-03-12T14:41:14-04:00,Desktop,3,review,We used this airline to go from Singapore to L...,Ok,0
1,2024-03-11T19:39:13-04:00,Desktop,5,review,The service on Singapore Airlines Suites Class...,The service in Suites Class makes one feel lik...,0
2,2024-03-11T12:20:23-04:00,Desktop,1,review,"Booked, paid and received email confirmation f...",Don’t give them your money,0
3,2024-03-11T07:12:27-04:00,Desktop,5,review,"Best airline in the world, seats, food, servic...",Best Airline in the World,0
4,2024-03-10T05:34:18-04:00,Desktop,2,review,Premium Economy Seating on Singapore Airlines ...,Premium Economy Seating on Singapore Airlines ...,0


In [3]:
data['rating'] = data['rating'].apply(lambda x: 1 if x > 3 else 0)
data['rating'].value_counts()

rating
1    7391
0    2609
Name: count, dtype: int64

In [4]:
data['review'] = data['title'] + ' ' + data['text']
data['review'] = data['review'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x) if x is not np.nan else '')
data = data.drop(['title', 'text', 'type', 'published_date', 'published_platform'], axis=1) # были мысли что - то сделать с helpful_votes, но потом решил что популярность отзыва никак не влияет на его позитивность или негативность
data.head()

,rating,helpful_votes,review
0,0,0,Ok We used this airline to go from Singapore t...
1,1,0,The service in Suites Class makes one feel lik...
2,0,0,Dont give them your money Booked paid and rece...
3,1,0,Best Airline in the World Best airline in the ...
4,0,0,Premium Economy Seating on Singapore Airlines ...


In [5]:
steamer = PorterStemmer()

def tokenize(txt):
    return [steamer.stem(word) for word in word_tokenize(txt.lower())]

ban = list(text.ENGLISH_STOP_WORDS)
vectorizer = TfidfVectorizer(stop_words=ban, tokenizer=tokenize)
X = vectorizer.fit_transform(data['review'])
X = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

X_train, X_test, y_train, y_test = train_test_split(X, data['rating'], test_size=0.2, random_state=42)

C:\Users\skfim\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\skfim\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'there

In [6]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

0.9378943872467618

In [7]:
model = SVC(max_iter=10) # вообще без указания max_iter показывала лучшие результаты, но обучалась невероятно долго, а перебор параметров вообще умер, стопнул после 3 часов
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

C:\Users\skfim\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


0.5913884007029877

In [8]:
model = CatBoostClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

Learning rate set to 0.025035
0:	learn: 0.6804341	total: 275ms	remaining: 4m 34s
1:	learn: 0.6680916	total: 399ms	remaining: 3m 19s
2:	learn: 0.6547196	total: 536ms	remaining: 2m 58s
3:	learn: 0.6406167	total: 677ms	remaining: 2m 48s
4:	learn: 0.6282128	total: 806ms	remaining: 2m 40s
5:	learn: 0.6171379	total: 932ms	remaining: 2m 34s
6:	learn: 0.6070113	total: 1.06s	remaining: 2m 30s
7:	learn: 0.5958746	total: 1.2s	remaining: 2m 29s
8:	learn: 0.5868300	total: 1.33s	remaining: 2m 26s
9:	learn: 0.5795183	total: 1.46s	remaining: 2m 25s
10:	learn: 0.5716834	total: 1.6s	remaining: 2m 23s
11:	learn: 0.5635444	total: 1.73s	remaining: 2m 22s
12:	learn: 0.5541184	total: 1.87s	remaining: 2m 21s
13:	learn: 0.5469784	total: 2.01s	remaining: 2m 21s
14:	learn: 0.5384779	total: 2.14s	remaining: 2m 20s
15:	learn: 0.5316063	total: 2.28s	remaining: 2m 20s
16:	learn: 0.5260488	total: 2.42s	remaining: 2m 20s
17:	learn: 0.5187715	total: 2.56s	remaining: 2m 19s
18:	learn: 0.5108978	total: 2.71s	remaining: 2

0.9361702127659574

In [9]:
# Вообще хотел перебирать параметры для svc, но комп не выдержал
# переберем параметры для логрега
params = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
}

grid = GridSearchCV(LogisticRegression(), params, cv=3, scoring='f1')
grid.fit(X_train, y_train)
grid.best_params_

C:\Users\skfim\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\skfim\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

{'C': 10, 'penalty': 'l2'}

In [12]:
review_pos = 'The best airline in the world. I have flown with Singapore Airlines for many years and have always been impressed with the service. The cabin crew are always friendly and helpful. The food is excellent and the seats are comfortable. I would highly recommend Singapore Airlines to anyone.'

review_neg = 'The worst airline in the world. I have flown with Singapore Airlines for many years and have always been disappointed with the service. The cabin crew are always rude and unhelpful. The food is terrible and the seats are uncomfortable. I would not recommend Singapore Airlines to anyone.'

review_pos = re.sub(r'[^a-zA-Z\s]', '', review_pos)

review_neg = re.sub(r'[^a-zA-Z\s]', '', review_neg)

review_pos = vectorizer.transform([review_pos])

review_neg = vectorizer.transform([review_neg])

final_model = LogisticRegression(C=10, penalty='l2')
final_model.fit(X, data['rating'])

print('pos' if final_model.predict(review_pos) == 1 else 'neg')
print('pos' if final_model.predict(review_neg) == 1 else 'neg')

pos
neg


C:\Users\skfim\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
